# Neutrino direction reconstruction


### Jan 7, 2021

## Code

In [1]:
import sys
import csv
# sys.path.insert(0,"/users/PCON0003/cond0068/.local/lib/python3.7/")
sys.path.append("/users/PAS0654/osu8354/ARA_cvmfs/root_build/lib/") # go to parent dir
sys.path.append("/users/PCON0003/cond0068/.local/lib/python3.7/site-packages/")
# sys.path.append("/users/PCON0003/cond0068/pyrex_sims/fromBen/thesis_work/pyrex-custom/analysis/custom/analysis/")
import ROOT
import math
import numpy as np
from ROOT import TH1D,TF1, gRandom, gPad, gStyle
import matplotlib as mpl
import matplotlib.pyplot as plt
from ROOT import TChain, TSelector, TTree
import os
import matplotlib.colors as mcolors
import scipy
from matplotlib.colors import LogNorm
import pandas as pd
import pyrex
import seaborn as sns
%matplotlib inline
sys.path.insert(1, "/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/CenA_sourceSearch/Stokes")
import deDisperse_util as util
from pyrex.internal_functions import normalize
my_path_plots = os.path.abspath("./plots/")
import pyrex.custom.ara as ara


Welcome to JupyROOT 6.18/04


In [2]:
# mpl.use('agg') 
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.unicode'] = True
mpl.rcParams['mathtext.rm'] = 'Times New Roman'
mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'

mpl.rc('font', family='serif', size=12)
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['ytick.major.size'] = 5

mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
# mpl.rc('font', size=16)
mpl.rc('axes', titlesize=20)

current_palette = sns.color_palette('colorblind', 10)
import warnings
warnings.filterwarnings("ignore")


In [3]:
def SphericalToCartesian(phi, theta):
    x = np.sin(theta)*np.cos(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(theta)
    
    return np.array([x,y,z])

#### Using output from AraSim at antenna

From https://arxiv.org/abs/1911.02093, the neutrino direction $\hat{v}$ can be obatined from the viewing angle $\theta$, the poarization vector $\hat{p}$. amd the launch vector $\hat{\ell}$ as follows:
$$
\hat{v} = \sin{\theta}\hat{p}-\cos{\theta}\hat{\ell}
$$

We can check that by using those quatities obtained from AraSim outputs at the antenna (this is importan, since some of these quantities might be in a different coordinate system)

The MC truth neutrino direction (in AraSim Earth-centered coordinates) is 
$$
\hat{v} = (-0.59808, -0.15246, -0.78679)
$$

In [4]:
polVec = np.array([0.024466,-0.023636,0.999421])
propVec = np.array([0.95729854,0.28862371,-0.01660877])
viewAngle = 0.85369
antennaPos = np.array([10004.71607,9989.46621,6359632.39013])
antennaPos = antennaPos/np.linalg.norm(antennaPos)

In [5]:
np.sin(viewAngle)*polVec-np.cos(viewAngle)*propVec #Not equal to nnu

array([-0.61070248, -0.20750006,  0.76418961])

In [6]:
np.degrees(np.arccos(propVec[2]))

90.95165617991329

#### Using MC from AraSim at source

In [7]:
polSrc = np.array([0.200234,0.0293387,0.979309])
launchVecSrc = np.array([-0.938089,-0.28263,0.200273])
viewAngleSrc = np.radians(63.476)

In [8]:
np.degrees(np.arccos(launchVecSrc[2]))

78.44707622096229

In [9]:
np.sin(viewAngleSrc)*polSrc-np.cos(viewAngleSrc)*launchVecSrc #equal to nnu, module a minus sign (TBD)

array([0.59808373, 0.15246555, 0.78679772])

#### This means that we need to change all coordinates to global coordinates, and make sure that the launch angle is the same as the receiving angle (I guess in the same coordinate system)

At the antenna, the receiving angle $\theta_{rec}=90.95$. At the source, the launching angle is $\theta_{\ell}=78.45$. They're then clearly different. If we assume null ray-beinding (not so crazy, since we're looking at direct solutions), they should be similar (if they were in the same coordinate system, of course)

## Try with another event

The MC truth neutrino direction (in AraSim Earth-centered coordinates) is 
$$
\hat{v} = (0.48432, 0.69414, 0.53253)
$$

#### Using output from AraSim at antenna

In [10]:
polVec = np.array([-0.775350,-0.623478,-0.100534])
propVec = -np.array([0.36815365, -0.31688518, -0.87409763])
viewAngle = 1.03656327604786
antennaPos = np.array([10004.71607,9989.46621,6359632.39013])
antennaPos = antennaPos/np.linalg.norm(antennaPos)
print("ViewAngle = %0.3f"%np.degrees(viewAngle))

ViewAngle = 59.391


In [11]:
np.sin(viewAngle)*polVec-np.cos(viewAngle)*propVec #Not equal to nnu

array([-0.47985538, -0.69795415, -0.53159953])

#### Using MC from AraSim at source

In [12]:
polSrc = np.array([-0.776197,-0.622749,-0.0984964])
launchVecSrc = np.array([-0.360816,0.310635,0.879385])
viewAngleSrc = np.radians(59.3907)

In [13]:
np.sin(viewAngleSrc)*polSrc-np.cos(viewAngleSrc)*launchVecSrc #equal to nnu, module a minus sign (TBD)

array([-0.48432054, -0.69414425, -0.53253809])

Well, it seems to work for this event, modulo a minus sign in the final vector. Let's try another event

## Try with another event

The MC truth neutrino direction (in AraSim Earth-centered coordinates) is 
$$
\hat{v} = (0.14837, 0.93127, -0.33273)
$$

#### Using output from AraSim at antenna

In [22]:
polVec = np.array([-0.616971,-0.708495,0.342611])
propVec = SphericalToCartesian(phi,theta)
viewAngle = 1.1392932
antennaPos = np.array([10004.71607,9989.46621,6359632.39013])
antennaPos = antennaPos/np.linalg.norm(antennaPos)
print("ViewAngle = %0.3f"%np.degrees(viewAngle))

ViewAngle = 65.404


In [23]:
np.sin(viewAngle)*polVec-np.cos(viewAngle)*propVec #Not equal to nnu

array([-0.2196768 , -0.88226344,  0.31998384])

#### Using MC from AraSim at source

In [29]:
polSrc = np.array([-0.54077,-0.761677,0.356955])
launchVecSrc = np.array([-0.819697,0.572437,-0.0203246])
viewAngleSrc = np.radians(65.2767)

In [30]:
np.sin(viewAngleSrc)*polSrc-np.cos(viewAngleSrc)*launchVecSrc #equal to nnu, module a minus sign (TBD)

array([-0.14837483, -0.93127434,  0.33273634])

In [25]:
phi = np.arctan2(propVec[1],propVec[0])
theta = np.arccos(launchVecSrc[2])
# 180-np.degrees(np.arccos(0.28209558))

In [26]:
SphericalToCartesian(phi,theta)

array([-0.82003937,  0.5719461 , -0.0203246 ])

In [24]:
np.degrees(np.arccos(antennaPos[2]))

0.12737338191164163